**MODELO GARCH**

**IMPORTANDO PAQUETES IMPORTANTES**

En este primera sección se realiza un la importación de paquetes que nos sesrvirán para hacer los respecitvos cálculos para este modelo. Primero, importamo la biblioteca de Python, pandas, para acceder a las herramientas de datos como DataFrames y Series de tiempo que se encuentra en el diferentes formatos, en nuestro caso un csv "Index 18-23". Luego utilizamos importamos paquetes para realizar operaciones lógicas en arreglos multidimensionales, crear gráficos y visualizaciones tanto estáticas como animadas. Además de funciones para graficar características de las series temporales como la autocorrelación y  utilizar herramientas estáticas para series temporales -test de estacionariedad-. Después cargamos el módulo que contiene a los modelos ARIMA y ARMA que nos permitirpa modelar y predecir series temporales usando el modelo Modelo Autoregresivo de Media Móvil. El comando de scipy al indicarle import chi2 consideramos la distribución chi-cuadrado. También, gracias a math nos proporciona funciones matemáticas como la función raíz cuadrada. Finalmente, seaborn tiene una interfaz de alto nivel para crear gráficos estadísticos atractivos.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2
!pip install arch
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 12.1 MB/s eta 0:00:00


**IMPORTACIÓN Y PREPROCESAMIENTO DE LA DATA**

Estos primeros comando sirven para leer el archico CSV llamdno "df_comp.cvs". Luego crea una copia independiente del dataframe y se almacena en la variable "df_comp". Se convierte la columna 'date' del DataFrame a un tipo de dato de fecha (datetime), asumiendo que el día aparece primero en la fecha. Se establece la columna 'date' como el índice del DataFrame df_comp. También se cambia la frecuencia del DataFrame a días hábiles, donde 'b' representa los días hábiles. Luego con el método "ffill" se completan los datos faltantes por "NaN". Lo siguiente es crear una columna llamada 'market_value' en el DataFrame con los valores de la columna 'ftse'. Posterior a ello, eliminamos las siguientes columnas del dataframe df_comp: 'spx', 'dax','ftse' y 'nikkei'. Ya para finalizar almacenamos el valor del 80% del número de filas en el DataFrame en la variable size. Finalmente, dividimos el DataFrame en dos partes basándose en la variable size. El DataFrame df contendrá las primeras filas hasta el índice size, mientras que df_test contendrá las filas restantes desde el índice size en adelante.

In [3]:
raw_csv_data = pd.read_csv("df_comp.csv")
df_comp=raw_csv_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst = True)
df_comp.set_index("date", inplace=True)
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

El siguiente comando duplica la columna llamada 'ftse' del DataFrame y lo renombra como 'market_value'.

In [4]:
df_comp['market_value']=df_comp.ftse

Los primeros cuatro comandos sirven para eliminar las columnas 'spx', 'dax', 'ftse' y 'nikkei' del DataFrame 'df_comp'. Luego, calcula el 80% del número de filas en df_comp y lo almaceno en la variable size como un número entero. Finalmente, divide el DataFrame df_comp en dos partes usando la técnica de segmentación con iloc.[:size]: Selecciona las filas desde el inicio hasta el índice size (sin incluir el índice size). [size:]: Selecciona las filas desde el índice size hasta el final del DataFrame. Luego, estas dos segmentaciones se asignan a dos nuevos DataFrames: df, que contiene el 80% de los datos y df_test, un conjunto de prueba que contiene el 20% restante de los datos.

In [5]:
del df_comp["ftse"]
del df_comp[" dax "]
del df_comp["spx "]
del df_comp["nikkei"]
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

Lo siguiente es importar el módulo warnings para advertir sobre cambios que puedan afectar su programa en futuras versiones de Python, o sobre prácticas que no son recomendables. Estas advertencias no detienen la ejecución del programa pero se muestran en la consola para informar al desarrollador. Al incluir "ignore", le decimos que no nos muestre las advertencia y las oculte.

In [6]:
import warnings
warnings.filterwarnings("ignore")

**TEST LLR (Likelihood Ratio)**

Este test permite comparar la bondad de ajuste de dos modelos, prueva de razón de verosimilitud. Contrasto la adecuación de un modelo restringido en comparación con un modelo más general (o no restringido).

Definimos una función llamada LLR_test que toma tres argumentos: mod_1, mod_2 y DF con un valor predeterminado de 1 grado de libertad. Luego se ajustan el modelo mod_1 y mod_2 usando el método fit() y luego se obtiene la función logarítmica de verosimilitud (log likelihood function) usando .llf. El valor resultante se almacena en la variable L1 y L2. Después, se calcula la razón de verosimilitud (LR), la cual es la diferencia entre las dos funciones logarítmicas de verosimilitud (L2 - L1) y la multiplica por 2. Finalmente, calculo el valor de p-value asociados al LR y el GL y se redonde su valor a 3 decimales.

In [7]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round(3)
    return p

**CREACIÓN DE RETORNOS**

Aquí se está accediendo a la columna market_value del DataFrame df. para calcular el cambio porcentual entre el elemento actual y al usar el argumento 1, calcula el cambio porcentual respecto al elemento anterior. Entonces con este comando se está creando la variable returns en el DataFrame df, y asignándole los valores del cambio porcentual calculado de la columna market_value.

In [8]:
df['returns'] = df.market_value.pct_change(1)*100

**MODELO GARCH SIMPLE**

In [9]:
model_garch_1_1 = arch_model(df.returns[1:], mean = "Constant", vol = "GARCH", p = 1, q = 1)
results_garch_1_1 = model_garch_1_1.fit(update_freq = 5)
results_garch_1_1.summary()

Iteration:      5,   Func. Count:     38,   Neg. LLF: 1409.5869366024106
Iteration:     10,   Func. Count:     64,   Neg. LLF: 1401.7748877256067
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1401.7748844130797
            Iterations: 12
            Function evaluations: 73
            Gradient evaluations: 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1401.77
Distribution:                  Normal   AIC:                           2811.55
Method:            Maximum Likelihood   BIC:                           2831.16
                                        No. Observations:                  995
Date:                Sat, Oct 14 2023   Df Residuals:                      994
Time:                        01:18:02   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu             0.0249  2.631e-02      0.945      0.345 [-2.671e-02,7.643e-02]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0666  2.447e-02      2.722  6.485e-03 [1.865e-02,  0.115]
alpha[1]       0.1451  4.132e-02      3.511  4.469e-04 [6.408e-02,  0.226]
beta[1]        0.8013  4.603e-02     17.406  7.383e-68   [  0.711,  0.892]
==========================================================================

Covariance estimator: robust
"""

**MODELOS GARCH CON MAYORES REZAGOS**

In [10]:
model_garch_1_2 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 2)
results_garch_1_2 = model_garch_1_2.fit(update_freq = 5)
results_garch_1_2.summary()

Iteration:      5,   Func. Count:     39,   Neg. LLF: 1459.038814609465
Iteration:     10,   Func. Count:     73,   Neg. LLF: 1400.5562463369524
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1400.5562142290696
            Iterations: 13
            Function evaluations: 90
            Gradient evaluations: 13


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1400.56
Distribution:                  Normal   AIC:                           2811.11
Method:            Maximum Likelihood   BIC:                           2835.63
                                        No. Observations:                  995
Date:                Sat, Oct 14 2023   Df Residuals:                      994
Time:                        01:18:08   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu             0.0235  2.617e-02      0.898      0.369 [-2.779e-02,7.480e-02]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0834  3.127e-02      2.667  7.653e-03 [2.211e-02,  0.145]
alpha[1]       0.1879  5.219e-02      3.601  3.172e-04 [8.564e-02,  0.290]
beta[1]        0.4033      0.130      3.099  1.940e-03   [  0.148,  0.658]
beta[2]        0.3419      0.116      2.950  3.173e-03   [  0.115,  0.569]
==========================================================================

Covariance estimator: robust
"""

In [11]:
model_garch_2_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 2, q = 1)
results_garch_2_1 = model_garch_2_1.fit(update_freq = 5)
results_garch_2_1.summary()

Iteration:      5,   Func. Count:     39,   Neg. LLF: 1433.3806314861254
Iteration:     10,   Func. Count:     73,   Neg. LLF: 1403.1371217490164
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1401.7748843856016
            Iterations: 14
            Function evaluations: 97
            Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1401.77
Distribution:                  Normal   AIC:                           2813.55
Method:            Maximum Likelihood   BIC:                           2838.06
                                        No. Observations:                  995
Date:                Sat, Oct 14 2023   Df Residuals:                      994
Time:                        01:18:12   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu             0.0249  2.624e-02      0.947      0.343 [-2.657e-02,7.629e-02]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0666  2.680e-02      2.485  1.296e-02 [1.407e-02,  0.119]
alpha[1]       0.1451  5.801e-02      2.501  1.239e-02 [3.138e-02,  0.259]
alpha[2]   5.0958e-14  5.865e-02  8.688e-13      1.000   [ -0.115,  0.115]
beta[1]        0.8013  5.209e-02     15.382  2.160e-53   [  0.699,  0.903]
==========================================================================

Covariance estimator: robust
"""

In [12]:
model_garch_3_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 3, q = 1)
results_garch_3_1 = model_garch_3_1.fit(update_freq = 5)
results_garch_3_1.summary()

Iteration:      5,   Func. Count:     43,   Neg. LLF: 1833.149478602304
Iteration:     10,   Func. Count:     84,   Neg. LLF: 1400.942377455051
Iteration:     15,   Func. Count:    118,   Neg. LLF: 1400.9313577527419
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1400.9313577529579
            Iterations: 15
            Function evaluations: 118
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1400.93
Distribution:                  Normal   AIC:                           2813.86
Method:            Maximum Likelihood   BIC:                           2843.28
                                        No. Observations:                  995
Date:                Sat, Oct 14 2023   Df Residuals:                      994
Time:                        01:18:17   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu             0.0252  2.640e-02      0.954      0.340 [-2.656e-02,7.693e-02]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          0.0796  3.440e-02      2.314  2.067e-02  [1.218e-02,  0.147]
alpha[1]       0.1214  5.239e-02      2.317  2.051e-02  [1.870e-02,  0.224]
alpha[2]   8.6048e-14  7.212e-02  1.193e-12      1.000    [ -0.141,  0.141]
alpha[3]       0.0467  6.565e-02      0.711      0.477 [-8.200e-02,  0.175]
beta[1]        0.7673  6.973e-02     11.005  3.616e-28    [  0.631,  0.904]
===========================================================================

Covariance estimator: robust
"""